DESCRIPCION

Este procedimiento almacenado: 
1. eliminará cualquier registro existente en la tabla "tbl_flujo_reporte" para el mes y año especificados y luego 
2. insertará nuevos registros basados en los acumulados de NETO por "ITEM_COD" para ese mes y año.

========================================================================================

In [1]:
import mysql.connector
import configparser

In [2]:
def actualizar_saldo_mes(p_mes, p_anio):
    try:
        # Leer las credenciales del archivo de configuración
        config = configparser.ConfigParser()
        config.read('config.ini')

        user = config['database']['user']
        password = config['database']['password']
        host = config['database']['host']
        database = config['database']['database']

        # Abrir la base de datos: establecer la conexión
        conexion = mysql.connector.connect(user=user, password=password, host=host, database=database)

        # crear un cursor
        cursor = conexion.cursor()

        # =====================================================================================================================

        # Obtener el mes y año del periodo anterior
        if p_mes == 1:
            v_mes_anterior = 12
            v_anio_anterior = p_anio - 1
        else:
            v_mes_anterior = p_mes - 1
            v_anio_anterior = p_anio

        # Verificar si existe un registro para el mes anterior
        cursor.execute("SELECT COUNT(*) FROM tbl_saldo_mes WHERE mes = %s AND anio = %s", (v_mes_anterior, v_anio_anterior))
        v_existencia_mes_anterior = cursor.fetchone()[0]

        if v_existencia_mes_anterior == 0:
            print("No hay registro de saldos para el mes anterior. Por favor, verifique los parámetros o corrija la situación")
            sys.exit()

        # Obtener la fecha del último día del mes
        cursor.execute("SELECT LAST_DAY('{}-{}-01')".format(p_anio, p_mes))
        v_fecha = cursor.fetchone()[0]

        # Verificar si ya existe un registro para este mes y año
        cursor.execute("SELECT COUNT(*) FROM tbl_saldo_mes WHERE mes = %s AND anio = %s", (p_mes, p_anio))
        v_existencia = cursor.fetchone()[0]

        if v_existencia > 0:
            # Obtener el saldo inicial del periodo anterior
            cursor.execute("SELECT saldo_final FROM tbl_saldo_mes WHERE mes = %s AND anio = %s", (v_mes_anterior, v_anio_anterior))
            v_saldo_inicial = cursor.fetchone()[0]

            # Obtener la sumatoria del campo NETO de tbl_transacciones
            cursor.execute("SELECT IFNULL(SUM(NETO), 0) FROM tbl_transacciones WHERE MES = %s AND ANIO = %s", (p_mes, p_anio))
            v_neto_mes = cursor.fetchone()[0]

            # Calcular el saldo final
            v_saldo_final = v_saldo_inicial + v_neto_mes

            # Actualizar el registro existente
            cursor.execute("UPDATE tbl_saldo_mes SET fecha = %s, saldo_inicial = %s, neto_mes = %s, saldo_final = %s WHERE mes = %s AND anio = %s", (v_fecha, v_saldo_inicial, v_neto_mes, v_saldo_final, p_mes, p_anio))
        else:
            # Obtener el saldo inicial del periodo anterior
            cursor.execute("SELECT saldo_final FROM tbl_saldo_mes WHERE mes = %s AND anio = %s", (v_mes_anterior, v_anio_anterior))
            v_saldo_inicial = cursor.fetchone()[0]

            # Obtener la sumatoria del campo NETO de tbl_transacciones
            cursor.execute("SELECT IFNULL(SUM(NETO), 0) FROM tbl_transacciones WHERE MES = %s AND ANIO = %s", (p_mes, p_anio))
            v_neto_mes = cursor.fetchone()[0]

            # Calcular el saldo final
            v_saldo_final = v_saldo_inicial + v_neto_mes

            # Insertar un nuevo registro
            cursor.execute("INSERT INTO tbl_saldo_mes (fecha, mes, anio, saldo_inicial, neto_mes, saldo_final) VALUES (%s, %s, %s, %s, %s, %s)", (v_fecha, p_mes, p_anio, v_saldo_inicial, v_neto_mes, v_saldo_final))

        # Confirmar los cambios
        conexion.commit()
        print("Proceso completado exitosamente.")

    except mysql.connector.Error as error:
        print("Error al conectar con la base de datos:", error)

    finally:
        # Cerrar la conexión
        if 'conexion' in locals() and conexion.is_connected():
            cursor.close()
            conexion.close()

# Ejemplo de uso
actualizar_saldo_mes(6, 2024)  # Especifica el mes y el año

No hay registro de saldos para el mes anterior. Por favor, verifique los parámetros o corrija la situación


NameError: name 'sys' is not defined

# Antes de ejecutar cualquier proceso validar que exista registro del periodo anterior
# Si no existe registro del periodo anterior, no crear ni actualizar registros

La tabla "tbl_flujo_reporte" debe resumir en cada "id_item" el contenido del campo NETO de la tabla "tbl_transacciones" según "ITEM_COD", "ANIO" y "MES"

Evaluar las siguientes opciones:
1. ir creando una columna en tabla "tbl_flujo_reporte" para resumir la información de cada mes/año
2. Modificar la estructura de la tabla "tbl_flujo_reporte" para que reciba un registro por cada acumulado de NETO por "ITEM_COD",   "ANIO" y "MES"

========================================================================================

1. **Crear una columna en la tabla "tbl_flujo_reporte" para resumir la información de cada mes/año**: 
* Esta opción implica agregar una nueva columna para cada mes/año en la tabla "tbl_flujo_reporte". Cada vez que se ingresen nuevos datos en la tabla "tbl_transacciones", tendrás que actualizar las columnas correspondientes en "tbl_flujo_reporte". 
* Esto podría ser eficiente si necesitas acceder rápidamente a los totales mensuales/año sin tener que realizar cálculos complejos en tiempo de consulta. 
* Sin embargo, podría volverse complicado si hay muchos meses/años diferentes o si la cantidad de datos es grande, ya que tendrías muchas columnas en la tabla "tbl_flujo_reporte".

2. **Modificar la estructura de la tabla "tbl_flujo_reporte" para que reciba un registro por cada acumulado de NETO por "ITEM_COD", "ANIO" y "MES"**: 
* En esta opción, en lugar de tener una columna para cada mes/año en "tbl_flujo_reporte", tendrías una fila por cada combinación de "ITEM_COD", "ANIO" y "MES", con una columna que almacena el total NETO correspondiente. 
* Esto sería más escalable y flexible, ya que no tendrías que modificar la estructura de la tabla cada vez que agregues un nuevo mes/año. 
* Además, permitiría consultas más flexibles y eficientes sobre los datos agregados.

En general, la opción 2 parece ser la más flexible y escalable, ya que te permite almacenar los totales NETO de manera estructurada y realizar consultas eficientes sobre ellos. Sin embargo, la elección final dependerá de tus necesidades específicas y de la complejidad de tu aplicación.

========================================================================================

Uno de los propósitos de la tabla "tbl_flujo_reporte" es que sirva como puente para tomar datos que alimentarán un reporte

* Si el propósito principal de la tabla "tbl_flujo_reporte" es alimentar un reporte, entonces la opción 2 parece ser la más adecuada. 
* Al tener una fila por cada combinación de "ITEM_COD", "ANIO" y "MES" con el total NETO correspondiente, puedes generar fácilmente los datos necesarios para el reporte utilizando consultas SQL simples.
* Además, al utilizar esta estructura, la tabla "tbl_flujo_reporte" actuará como un puente eficiente para obtener los datos agregados necesarios para el reporte, ya que los totales NETO estarán precalculados y disponibles en la tabla, lo que evitará la necesidad de realizar cálculos complejos en tiempo de consulta.

Por lo tanto, la opción 2 te proporcionará una estructura eficiente y escalable para alimentar el reporte, lo que facilitará el proceso de generación del mismo y garantizará un rendimiento óptimo, especialmente si estás trabajando con grandes volúmenes de datos.

Consulta para mostrar los procedimientos almacenados (stored procedures) en MySQL:

SHOW PROCEDURE STATUS;

========================================================================================

Consulta que muestra el código fuente completo del procedimiento almacenado (lógica y parámetros):

SHOW CREATE PROCEDURE nombre_del_procedimiento;

========================================================================================

Borrar:

DROP PROCEDURE IF EXISTS nombre_del_procedimiento;

========================================================================================